In [ ]:
import multiprocessing
import os
from re import A
from typing import ItemsView

from py import log
os.environ['PYTHONDONTWRITEBYTECODE'] = '1'
from xtquant import xttrader
from xtquant import xtdata
from xtquant.xttrader import XtQuantTrader, XtQuantTraderCallback
from xtquant.xttype import StockAccount
from xtquant import xtconstant
import pandas_market_calendars as mcal
import pandas as pd
import akshare as ak

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.dates as mdates

import sqlite3

def plot(df, column):
    
    # 设置中文字体
    mpl.rcParams['font.sans-serif'] = ['SimHei']  # 指定中文字体为 SimHei
    mpl.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

    # 假设这是你的DataFrame，其中包含收益

    # 绘制收益曲线
    # 将日期列转换为日期类型
    # df['date'] = pd.to_datetime(df['date'])

    # # 设置日期列为索引
    # df.set_index('date', inplace=True)
    df.index = pd.to_datetime(df.index)
    # 绘制收益曲线
    plt.figure(figsize=(10, 6))  # 设置图表大小
    plt.plot(df.index, df[column], label='Return')
    plt.title('收益曲线')
    plt.xlabel('日期')
    plt.ylabel('收益率')
    plt.legend()  # 显示图例
    plt.grid(True)  # 显示网格线

    # plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    # plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=10))

    plt.show()




def caculate_returns(returns_df, row, _print = False):
    r = {}
    cumulative_returns = (1 + returns_df[row]).cumprod()

    # 计算最大回撤
    cumulative_max = cumulative_returns.cummax()
    drawdown = (cumulative_returns - cumulative_max) / cumulative_max
    max_drawdown = drawdown.min()

    # 计算夏普比率
    risk_free_rate = 0.0  # 假设无风险利率为0
    sharpe_ratio = (returns_df[row].mean() - risk_free_rate) / returns_df[row].std()

    # 计算总收益率
    total_return = cumulative_returns.iloc[-1] - 1

    # 计算波动率
    volatility = returns_df[row].std()

    # 计算总盈亏
    total_profit_loss = cumulative_returns.iloc[-1] - cumulative_returns.iloc[0]

    # 计算成功次数、胜率、平均盈利、平均亏损、最大盈利、最大亏损以及盈亏比
    profitable_trades = returns_df[row][returns_df[row] > 0].count()
    losing_trades = returns_df[row][returns_df[row] < 0].count()
    win_rate = profitable_trades / (profitable_trades + losing_trades) if (profitable_trades + losing_trades) > 0 else 0
    average_profit = returns_df[row][returns_df[row] > 0].mean() if profitable_trades > 0 else 0
    average_loss = returns_df[row][returns_df[row] < 0].mean() if losing_trades > 0 else 0
    max_profit = returns_df[row].max()
    max_loss = returns_df[row].min()
    total_trades = profitable_trades + losing_trades
    total_times = returns_df[row].count()
    trade_pct = total_times / total_trades
    profit_loss_ratio = average_profit / abs(average_loss) if average_loss != 0 else 0
    kelly_fraction = win_rate - ((1 - win_rate) / profit_loss_ratio) if profit_loss_ratio != 0 else 0
    return_per_day = total_return / total_times
    year_return = return_per_day * 240

    r['最大回撤'] =  max_drawdown
    r['夏普比率'] =  sharpe_ratio
    r['总收益率'] =  total_return
    r['波动率'] = volatility
    r['年化收益率'] = year_return
    r['总盈亏'] = total_profit_loss
    r['成功次数'] = profitable_trades
    r['失败次数'] = losing_trades
    r['总天数'] = total_times
    r['总交易次数'] = total_trades
    r['交易频率'] = trade_pct
    r['胜率'] = win_rate
    r['平均盈利'] = average_profit
    r['平均亏损'] = average_loss
    r['最大盈利'] = max_profit
    r['最大亏损'] = max_loss
    r['盈亏比'] = profit_loss_ratio
    r['凯利公式最佳仓位'] = kelly_fraction
    if _print:
        print(f"最大回撤: {max_drawdown:.2%}")
        print(f"夏普比率: {sharpe_ratio:.2f}")
        print(f"总收益率: {total_return:.2%}")
        print(f"年化收益率: {year_return:.2%}")
        print(f"波动率: {volatility:.2%}")
        print(f"总盈亏: {total_profit_loss:.2%}")
        print(f"成功次数: {profitable_trades}")
        print(f"失败次数: {losing_trades}")
        print(f"总天数: {total_times}")
        print(f"总交易次数: {total_trades}")
        print(f"交易频率: {trade_pct:.2%}")
        print(f"胜率: {win_rate:.2%}")
        print(f"平均盈利: {average_profit:.2%}")
        print(f"平均亏损: {average_loss:.2%}")
        print(f"最大盈利: {max_profit:.2%}")
        print(f"最大亏损: {max_loss:.2%}")
        print(f"盈亏比: {profit_loss_ratio:.2f}")
        print(f"凯利公式最佳仓位: {kelly_fraction:.2%}")

    return r


# strategy_name = '低吸'

# sub_strategy_name = '低位N字低吸'

max_stock_rank = 10

xiaocao_fangxiang_filter = True

months = [ '202409', '202410', '202411', '202412', '202501', '202502' ]

# months = ['202501', '202502' ]

hd_pct = 0.003

top_fx =2


def group_filter_fx(group, filtered = True, fx_filtered = True, topn = 3):
    if not filtered:
        valid_rows = group[(group['open_price'] > 0) & (group['next_day_open_price'] > 0) & (group['stock_rank'] <= topn) & (group['next_day_close_price'] > 0)]
        if len(valid_rows) > 0:
            valid_rows['return'] = valid_rows['next_day_open_price'] / valid_rows['open_price'] - 1
            valid_rows['real_return'] = valid_rows['return'] - hd_pct
            valid_rows['close_return'] = valid_rows['next_day_close_price'] / valid_rows['open_price'] - 1
            valid_rows['close_real_return'] = valid_rows['close_return'] - hd_pct
            avg_value = valid_rows['return'].mean()
            close_avg_value = valid_rows['close_return'].mean()
            rank_one_row = group[group['stock_rank'] == 1].copy()
            if len(rank_one_row) > 0:
                # 将平均值赋给 rank 为 1 的行的指定列
                rank_one_row['return'] = avg_value
                rank_one_row['real_return'] = avg_value - hd_pct
                rank_one_row['close_return'] = close_avg_value
                rank_one_row['close_real_return'] = close_avg_value - hd_pct
                return rank_one_row
        else:
            rank_one_row = group[group['stock_rank'] == 1].copy()
            if len(rank_one_row) > 0:
                rank_one_row['return'] = -10
                rank_one_row['real_return'] = -10
                rank_one_row['close_return'] = -10
                rank_one_row['close_real_return'] = -10
                return rank_one_row
    else:
        if fx_filtered:
            min_category_rank = group['max_block_category_rank'].min()

            # block_rank_one_row = group[group['max_block_code_rank'] == 1].copy()

            # if not block_rank_one_row.empty and len(block_rank_one_row) == 1:
            #     block_rank_one_row['return'] = block_rank_one_row['next_day_open_price'] / block_rank_one_row['open_price'] - 1
            #     block_rank_one_row['real_return'] = block_rank_one_row['return'] - hd_pct
            #     block_rank_one_row['close_return'] = block_rank_one_row['next_day_close_price'] / block_rank_one_row['open_price'] - 1
            #     block_rank_one_row['close_real_return'] = block_rank_one_row['close_return'] - hd_pct
            #     return block_rank_one_row
            # elif not block_rank_one_row.empty and len(block_rank_one_row) > 1:
            #     block_rank_one_row = block_rank_one_row[block_rank_one_row['stock_rank'] == block_rank_one_row['stock_rank'].min()]
            #     block_rank_one_row['return'] = block_rank_one_row['next_day_open_price'] / block_rank_one_row['open_price'] - 1
            #     block_rank_one_row['real_return'] = block_rank_one_row['return'] - hd_pct
            #     block_rank_one_row['close_return'] = block_rank_one_row['next_day_close_price'] / block_rank_one_row['open_price'] - 1
            #     block_rank_one_row['close_real_return'] = block_rank_one_row['close_return'] - hd_pct
            #     return block_rank_one_row
            
            # industry_rank_one_row = group[group['max_industry_code_rank'] == 1].copy()
            # if not industry_rank_one_row.empty and len(industry_rank_one_row) == 1:
            #     industry_rank_one_row['return'] = industry_rank_one_row['next_day_open_price'] / industry_rank_one_row['open_price'] - 1
            #     industry_rank_one_row['real_return'] = industry_rank_one_row['return'] - hd_pct
            #     industry_rank_one_row['close_return'] = industry_rank_one_row['next_day_close_price'] / industry_rank_one_row['open_price'] - 1
            #     industry_rank_one_row['close_real_return'] = industry_rank_one_row['close_return'] - hd_pct
            #     return industry_rank_one_row
            # elif not industry_rank_one_row.empty and len(industry_rank_one_row) > 1:
            #     industry_rank_one_row = industry_rank_one_row[industry_rank_one_row['stock_rank'] == industry_rank_one_row['stock_rank'].min()]
            #     industry_rank_one_row['return'] = industry_rank_one_row['next_day_open_price'] / industry_rank_one_row['open_price'] - 1
            #     industry_rank_one_row['real_return'] = industry_rank_one_row['return'] - hd_pct
            #     industry_rank_one_row['close_return'] = industry_rank_one_row['next_day_close_price'] / industry_rank_one_row['open_price'] - 1
            #     industry_rank_one_row['close_real_return'] = industry_rank_one_row['close_return'] - hd_pct
            #     return industry_rank_one_row
            
            if min_category_rank > 2:
                rank_one_row = group[group['stock_rank'] == 1].copy()
                if len(rank_one_row) > 0:
                    rank_one_row['return'] = rank_one_row['next_day_open_price'] / rank_one_row['open_price'] - 1
                    rank_one_row['real_return'] = rank_one_row['return'] - hd_pct
                    rank_one_row['close_return'] = rank_one_row['next_day_close_price'] / rank_one_row['open_price'] - 1
                    rank_one_row['close_real_return'] = rank_one_row['close_return'] - hd_pct
                    return rank_one_row
            else:
                category_filtered = group[group['max_block_category_rank'] <= min_category_rank + 2]
                result = category_filtered[category_filtered['max_block_code_rank'] == category_filtered['max_block_code_rank'].min()]
                if len(result) > 1:
                    result = result[result['stock_rank'] == result['stock_rank'].min()]
                result['return'] = result['next_day_open_price'] / result['open_price'] - 1
                result['real_return'] = result['return'] - hd_pct
                result['close_return'] = result['next_day_close_price'] / result['open_price'] - 1
                result['close_real_return'] = result['close_return'] - hd_pct
                return result
        else:
            rank_one_row = group[group['stock_rank'] == 1].copy()
            if not rank_one_row.empty and len(rank_one_row) > 0:
                rank_one_row['return'] = rank_one_row['next_day_open_price'] / rank_one_row['open_price'] - 1
                rank_one_row['real_return'] = rank_one_row['return'] - hd_pct
                rank_one_row['close_return'] = rank_one_row['next_day_close_price'] / rank_one_row['open_price'] - 1
                rank_one_row['close_real_return'] = rank_one_row['close_return'] - hd_pct
            return rank_one_row






def group_filter(group, filtered = True, fx_filtered = True, topn = 3):
    if not filtered:
        valid_rows = group[(group['open_price'] > 0) & (group['next_day_open_price'] > 0) & (group['stock_rank'] <= topn) & (group['next_day_close_price'] > 0)]
        if len(valid_rows) > 0:
            valid_rows['return'] = valid_rows['next_day_open_price'] / valid_rows['open_price'] - 1
            valid_rows['real_return'] = valid_rows['return'] - hd_pct
            valid_rows['close_return'] = valid_rows['next_day_close_price'] / valid_rows['open_price'] - 1
            valid_rows['close_real_return'] = valid_rows['close_return'] - hd_pct
            avg_value = valid_rows['return'].mean()
            close_avg_value = valid_rows['close_return'].mean()
            rank_one_row = group[group['stock_rank'] == 1].copy()
            if len(rank_one_row) > 0:
                # 将平均值赋给 rank 为 1 的行的指定列
                rank_one_row['return'] = avg_value
                rank_one_row['real_return'] = avg_value - hd_pct
                rank_one_row['close_return'] = close_avg_value
                rank_one_row['close_real_return'] = close_avg_value - hd_pct
                return rank_one_row
        else:
            rank_one_row = group[group['stock_rank'] == 1].copy()
            if len(rank_one_row) > 0:
                rank_one_row['return'] = -10
                rank_one_row['real_return'] = -10
                rank_one_row['close_return'] = -10
                rank_one_row['close_real_return'] = -10
                return rank_one_row
    else:
        if fx_filtered:
            min_category_rank = group['max_block_category_rank'].min()

            block_rank_one_row = group[group['max_block_code_rank'] == 1].copy()

            if not block_rank_one_row.empty and len(block_rank_one_row) == 1:
                block_rank_one_row['return'] = block_rank_one_row['next_day_open_price'] / block_rank_one_row['open_price'] - 1
                block_rank_one_row['real_return'] = block_rank_one_row['return'] - hd_pct
                block_rank_one_row['close_return'] = block_rank_one_row['next_day_close_price'] / block_rank_one_row['open_price'] - 1
                block_rank_one_row['close_real_return'] = block_rank_one_row['close_return'] - hd_pct
                return block_rank_one_row
            elif not block_rank_one_row.empty and len(block_rank_one_row) > 1:
                block_rank_one_row = block_rank_one_row[block_rank_one_row['stock_rank'] == block_rank_one_row['stock_rank'].min()]
                block_rank_one_row['return'] = block_rank_one_row['next_day_open_price'] / block_rank_one_row['open_price'] - 1
                block_rank_one_row['real_return'] = block_rank_one_row['return'] - hd_pct
                block_rank_one_row['close_return'] = block_rank_one_row['next_day_close_price'] / block_rank_one_row['open_price'] - 1
                block_rank_one_row['close_real_return'] = block_rank_one_row['close_return'] - hd_pct
                return block_rank_one_row
            
            industry_rank_one_row = group[group['max_industry_code_rank'] == 1].copy()
            if not industry_rank_one_row.empty and len(industry_rank_one_row) == 1:
                industry_rank_one_row['return'] = industry_rank_one_row['next_day_open_price'] / industry_rank_one_row['open_price'] - 1
                industry_rank_one_row['real_return'] = industry_rank_one_row['return'] - hd_pct
                industry_rank_one_row['close_return'] = industry_rank_one_row['next_day_close_price'] / industry_rank_one_row['open_price'] - 1
                industry_rank_one_row['close_real_return'] = industry_rank_one_row['close_return'] - hd_pct
                return industry_rank_one_row
            elif not industry_rank_one_row.empty and len(industry_rank_one_row) > 1:
                industry_rank_one_row = industry_rank_one_row[industry_rank_one_row['stock_rank'] == industry_rank_one_row['stock_rank'].min()]
                industry_rank_one_row['return'] = industry_rank_one_row['next_day_open_price'] / industry_rank_one_row['open_price'] - 1
                industry_rank_one_row['real_return'] = industry_rank_one_row['return'] - hd_pct
                industry_rank_one_row['close_return'] = industry_rank_one_row['next_day_close_price'] / industry_rank_one_row['open_price'] - 1
                industry_rank_one_row['close_real_return'] = industry_rank_one_row['close_return'] - hd_pct
                return industry_rank_one_row
            
            if min_category_rank > top_fx:
                rank_one_row = group[group['stock_rank'] == 1].copy()
                if len(rank_one_row) > 0:
                    rank_one_row['return'] = rank_one_row['next_day_open_price'] / rank_one_row['open_price'] - 1
                    rank_one_row['real_return'] = rank_one_row['return'] - hd_pct
                    rank_one_row['close_return'] = rank_one_row['next_day_close_price'] / rank_one_row['open_price'] - 1
                    rank_one_row['close_real_return'] = rank_one_row['close_return'] - hd_pct
                    return rank_one_row
            elif min_category_rank < 0:
                category_filtered = group[(group['max_block_category_rank'] > 0) & (group['max_block_category_rank'] <= top_fx) & (group['max_block_code_rank'] <= top_fx) & (group['max_block_code_rank'] > 0)]
                if not category_filtered.empty and len(category_filtered) > 0:
                    category_filtered = category_filtered[category_filtered['stock_rank'] == category_filtered['stock_rank'].min()]
                    category_filtered['return'] = category_filtered['next_day_open_price'] / category_filtered['open_price'] - 1
                    category_filtered['real_return'] = category_filtered['return'] - hd_pct
                    category_filtered['close_return'] = category_filtered['next_day_close_price'] / category_filtered['open_price'] - 1
                    category_filtered['close_real_return'] = category_filtered['close_return'] - hd_pct
                    return category_filtered
                else:
                    rank_one_row = group[group['stock_rank'] == 1].copy()
                    if not rank_one_row.empty and len(rank_one_row) > 0:
                        rank_one_row['return'] = rank_one_row['next_day_open_price'] / rank_one_row['open_price'] - 1
                        rank_one_row['real_return'] = rank_one_row['return'] - hd_pct
                        rank_one_row['close_return'] = rank_one_row['next_day_close_price'] / rank_one_row['open_price'] - 1
                        rank_one_row['close_real_return'] = rank_one_row['close_return'] - hd_pct
                        return rank_one_row
                    else:
                        raise Exception("No data.")
            else:
                category_filtered = group[group['max_block_category_rank'] <= min_category_rank + 2]
                mbcr = category_filtered['max_block_code_rank'].min()
                if mbcr < 0:
                    result = category_filtered[(category_filtered['max_block_code_rank'] > 0) & (category_filtered['max_block_code_rank'] <= top_fx)]
                    if result.empty or len(result) < 1:
                        result = category_filtered[category_filtered['stock_rank'] == category_filtered['stock_rank'].min()]
                else:
                    result = category_filtered[category_filtered['max_block_code_rank'] == category_filtered['max_block_code_rank'].min()]
                if result.empty or len(result) < 1:
                    result = category_filtered[category_filtered['stock_rank'] == category_filtered['stock_rank'].min()]
                if len(result) > 1:
                    result = result[result['stock_rank'] == result['stock_rank'].min()]
                result['return'] = result['next_day_open_price'] / result['open_price'] - 1
                result['real_return'] = result['return'] - hd_pct
                result['close_return'] = result['next_day_close_price'] / result['open_price'] - 1
                result['close_real_return'] = result['close_return'] - hd_pct
                return result
        else:
            rank_one_row = group[group['stock_rank'] == 1].copy()
            if not rank_one_row.empty and len(rank_one_row) > 0:
                rank_one_row['return'] = rank_one_row['next_day_open_price'] / rank_one_row['open_price'] - 1
                rank_one_row['real_return'] = rank_one_row['return'] - hd_pct
                rank_one_row['close_return'] = rank_one_row['next_day_close_price'] / rank_one_row['open_price'] - 1
                rank_one_row['close_real_return'] = rank_one_row['close_return'] - hd_pct
            return rank_one_row
        


if __name__ == '__main__':

    import yaml
    file_name = r'D:\workspace\TradeX\ezMoney\roll_back.yml'
    with open(file_name, 'r',  encoding='utf-8') as file:
        config = yaml.safe_load(file)
        if config is None or 'configs' not in config:
            print("Config No data Error.")

    configs = config['configs']

    m = {}

    for config in configs:
       strategy_name=config['strategy_name']
       sub_tasks = config['sub_tasks']
       if not sub_tasks:
           continue
       for sub_task in sub_tasks:
            sub_strategy_name = sub_task['name']
            if sub_strategy_name:
                if strategy_name in m:
                    m[strategy_name].append(sub_strategy_name)
                else:
                    m[strategy_name] = []
                    m[strategy_name].append(sub_strategy_name)

    for strategy_name, sub_strategy_names in m.items():
        for sub_strategy_name in sub_strategy_names:
            if sub_strategy_name != '中位低吸':
                continue
            print(f"strategy_name: {strategy_name}, sub_strategy_name: {sub_strategy_name}")
            for i in range(0, len(months)):
                if i < len(months) - 1:
                    continue

                combined_df = pd.DataFrame()
                for month in months[i:]:
                    conn = sqlite3.connect('D:\workspace\TradeX\ezMoney\sqlite_db\strategy_data.db')
                    db_name = 'strategy_data_aftermarket_%s' % month
                    query = "select * from %s where (strategy_name = '%s' and sub_strategy_name = '%s' and stock_rank <= %s) " % (db_name, strategy_name, sub_strategy_name, max_stock_rank)
                    df = pd.read_sql_query(query, conn)
                    combined_df = pd.concat([combined_df, df], axis=0)
                combined_df = combined_df.reset_index(drop=True)

                if len(combined_df) <= 1:
                    continue

                df = combined_df.groupby(['date_key','strategy_name','sub_strategy_name']).apply(group_filter, filtered = True, fx_filtered = True).reset_index(drop=True)
                df_fx = combined_df.groupby(['date_key','strategy_name','sub_strategy_name']).apply(group_filter_fx, filtered = True, fx_filtered = True).reset_index(drop=True)

d:\anaconda3\envs\python3.8\lib\site-packages\exchange_calendars\exchange_calendar.py:2345: FutureWarning: 'T' is deprecated and will be removed in a future version. Please use 'min' instead of 'T'.
  align: pd.Timedelta | str = pd.Timedelta(1, "T"),


strategy_name: 低吸, sub_strategy_name: 中位低吸


C:\Users\Administrator\AppData\Local\Temp\ipykernel_28924\364619939.py:409: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = combined_df.groupby(['date_key','strategy_name','sub_strategy_name']).apply(group_filter, filtered = True, fx_filtered = True).reset_index(drop=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_28924\364619939.py:410: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_fx = com

In [2]:
df

,id,date_key,strategy_name,sub_strategy_name,stock_code,stock_name,stock_rank,block_category,block_codes,industry_code,...,mod_trend_score,mod_trend_score_change,mod_trend_rank,env_json_info,block_category_info,created_at,return,real_return,close_return,close_real_return
0,2234,2025-02-05,低吸,中位低吸,002917.XSHE,金奥博,2,"000012.BKDL,000023.BKDL,000018.BKDL,000035.BKD...","885743.DDBK,885517.DDBK,885700.DDBK,885619.DDB...",980364.ZHBK,...,-83.43,-5.75,55,"{""9A0001"": {""realShortLineScore"": -2.23, ""real...","{""000034.BKDL"": {""categoryCode"": ""000034.BKDL""...",2025-02-28 17:17:10,-0.069632,-0.072632,-0.021921,-0.024921
1,2277,2025-02-06,低吸,中位低吸,002530.XSHE,金财互联,3,"000012.BKDL,000003.BKDL,000038.BKDL,000026.BKD...","885757.DDBK,885517.DDBK,885980.DDBK,886019.DDB...","980365.ZHBK,980753.ZHBK,980366.ZHBK",...,-96.17,-1.52,63,"{""9A0001"": {""realShortLineScore"": 35.17, ""real...","{""000031.BKDL"": {""categoryCode"": ""000031.BKDL""...",2025-02-28 17:18:08,0.066608,0.063608,0.071867,0.068867
2,2337,2025-02-07,低吸,中位低吸,003005.XSHE,竞业达,3,"000012.BKDL,000003.BKDL,000038.BKDL,000008.BKD...","885514.DDBK,886062.DDBK,885586.DDBK,885694.DDB...","980753.ZHBK,980366.ZHBK",...,-84.58,-3.23,64,"{""9A0001"": {""realShortLineScore"": 49.57, ""real...","{""000027.BKDL"": {""categoryCode"": ""000027.BKDL""...",2025-02-28 17:19:05,0.204433,0.201433,0.166812,0.163812
3,2387,2025-02-10,低吸,中位低吸,000681.XSHE,视觉中国,2,"000012.BKDL,000003.BKDL,000038.BKDL,000026.BKD...","885933.DDBK,885757.DDBK,886019.DDBK,886041.DDB...","980355.ZHBK,980348.ZHBK",...,-100.50,-24.84,71,"{""9A0001"": {""realShortLineScore"": 52.05, ""real...","{""000031.BKDL"": {""categoryCode"": ""000031.BKDL""...",2025-02-28 17:19:57,0.001961,-0.001039,-0.009412,-0.012412
4,2455,2025-02-11,低吸,中位低吸,001368.XSHE,通达创智,4,"000021.BKDL,000012.BKDL,000026.BKDL,000018.BKD...","885899.DDBK,885840.DDBK,885598.DDBK,885617.DDB...",980373.ZHBK,...,-73.76,16.84,68,"{""9A0001"": {""realShortLineScore"": 54.81, ""real...","{""000003.BKDL"": {""categoryCode"": ""000003.BKDL""...",2025-02-28 17:20:53,-0.023214,-0.026214,-0.010714,-0.013714
5,2525,2025-02-12,低吸,中位低吸,000034.XSHE,神州数码,2,"000012.BKDL,000003.BKDL,000038.BKDL,000026.BKD...","885459.DDBK,886048.DDBK,885757.DDBK,885957.DDB...","980345.ZHBK,980366.ZHBK",...,-74.40,2.38,67,"{""9A0001"": {""realShortLineScore"": 56.55, ""real...","{""000003.BKDL"": {""categoryCode"": ""000003.BKDL""...",2025-02-28 17:21:52,0.046320,0.043320,0.131672,0.128672
6,2578,2025-02-13,低吸,中位低吸,002757.XSHE,南兴股份,3,"000003.BKDL,000026.BKDL,000023.BKDL,000006.BKD...","885459.DDBK,885930.DDBK,885957.DDBK,885517.DDB...",980365.ZHBK,...,-68.31,-2.57,67,"{""9A0001"": {""realShortLineScore"": 40.07, ""real...","{""000012.BKDL"": {""categoryCode"": ""000012.BKDL""...",2025-02-28 17:22:47,0.008969,0.005969,0.066143,0.063143
7,2605,2025-02-14,低吸,中位低吸,002036.XSHE,联创电子,2,"000012.BKDL,000038.BKDL,000014.BKDL,000006.BKD...","886048.DDBK,885467.DDBK,885545.DDBK,885675.DDB...",980357.ZHBK,...,-74.52,-1.02,68,"{""9A0001"": {""realShortLineScore"": 27.53, ""real...","{""000003.BKDL"": {""categoryCode"": ""000003.BKDL""...",2025-02-28 17:23:40,0.091590,0.088590,0.105745,0.102745
8,2649,2025-02-17,低吸,中位低吸,002031.XSHE,巨轮智能,1,"000038.BKDL,000018.BKDL,000006.BKDL,000034.BKD...","885930.DDBK,885694.DDBK,885502.DDBK,885517.DDB...",980365.ZHBK,...,-54.96,12.25,67,"{""9A0001"": {""realShortLineScore"": 28.81, ""real...","{""000003.BKDL"": {""categoryCode"": ""000003.BKDL""...",2025-02-28 17:24:41,0.092105,0.089105,0.063158,0.060158
9,2714,2025-02-18,低吸,中位低吸,002908.XSHE,德生科技,1,"000012.BKDL,000003.BKDL,000038.BKDL,000010.BKD...","885975.DDBK,885757.DDBK,886041.DDBK,885629.DDB...","980753.ZHBK,980366.ZHBK,980376.ZHBK",...,-34.04,-2.67,57,"{""9A0001"": {""realShortLineScore"": 7.77, ""realT...","{""000004.BKDL"": {""categoryCode"": ""000004.BKDL""...",2025-02-28 17:25:40,-0.021761,-0.024761,0.046489,0.043489


In [3]:
df_fx

,id,date_key,strategy_name,sub_strategy_name,stock_code,stock_name,stock_rank,block_category,block_codes,industry_code,...,mod_trend_score,mod_trend_score_change,mod_trend_rank,env_json_info,block_category_info,created_at,return,real_return,close_return,close_real_return
0,2234,2025-02-05,低吸,中位低吸,002917.XSHE,金奥博,2,"000012.BKDL,000023.BKDL,000018.BKDL,000035.BKD...","885743.DDBK,885517.DDBK,885700.DDBK,885619.DDB...",980364.ZHBK,...,-83.43,-5.75,55,"{""9A0001"": {""realShortLineScore"": -2.23, ""real...","{""000034.BKDL"": {""categoryCode"": ""000034.BKDL""...",2025-02-28 17:17:10,-0.069632,-0.072632,-0.021921,-0.024921
1,2277,2025-02-06,低吸,中位低吸,002530.XSHE,金财互联,3,"000012.BKDL,000003.BKDL,000038.BKDL,000026.BKD...","885757.DDBK,885517.DDBK,885980.DDBK,886019.DDB...","980365.ZHBK,980753.ZHBK,980366.ZHBK",...,-96.17,-1.52,63,"{""9A0001"": {""realShortLineScore"": 35.17, ""real...","{""000031.BKDL"": {""categoryCode"": ""000031.BKDL""...",2025-02-28 17:18:08,0.066608,0.063608,0.071867,0.068867
2,2337,2025-02-07,低吸,中位低吸,003005.XSHE,竞业达,3,"000012.BKDL,000003.BKDL,000038.BKDL,000008.BKD...","885514.DDBK,886062.DDBK,885586.DDBK,885694.DDB...","980753.ZHBK,980366.ZHBK",...,-84.58,-3.23,64,"{""9A0001"": {""realShortLineScore"": 49.57, ""real...","{""000027.BKDL"": {""categoryCode"": ""000027.BKDL""...",2025-02-28 17:19:05,0.204433,0.201433,0.166812,0.163812
3,2387,2025-02-10,低吸,中位低吸,000681.XSHE,视觉中国,2,"000012.BKDL,000003.BKDL,000038.BKDL,000026.BKD...","885933.DDBK,885757.DDBK,886019.DDBK,886041.DDB...","980355.ZHBK,980348.ZHBK",...,-100.50,-24.84,71,"{""9A0001"": {""realShortLineScore"": 52.05, ""real...","{""000031.BKDL"": {""categoryCode"": ""000031.BKDL""...",2025-02-28 17:19:57,0.001961,-0.001039,-0.009412,-0.012412
4,2455,2025-02-11,低吸,中位低吸,001368.XSHE,通达创智,4,"000021.BKDL,000012.BKDL,000026.BKDL,000018.BKD...","885899.DDBK,885840.DDBK,885598.DDBK,885617.DDB...",980373.ZHBK,...,-73.76,16.84,68,"{""9A0001"": {""realShortLineScore"": 54.81, ""real...","{""000003.BKDL"": {""categoryCode"": ""000003.BKDL""...",2025-02-28 17:20:53,-0.023214,-0.026214,-0.010714,-0.013714
5,2525,2025-02-12,低吸,中位低吸,000034.XSHE,神州数码,2,"000012.BKDL,000003.BKDL,000038.BKDL,000026.BKD...","885459.DDBK,886048.DDBK,885757.DDBK,885957.DDB...","980345.ZHBK,980366.ZHBK",...,-74.40,2.38,67,"{""9A0001"": {""realShortLineScore"": 56.55, ""real...","{""000003.BKDL"": {""categoryCode"": ""000003.BKDL""...",2025-02-28 17:21:52,0.046320,0.043320,0.131672,0.128672
6,2578,2025-02-13,低吸,中位低吸,002757.XSHE,南兴股份,3,"000003.BKDL,000026.BKDL,000023.BKDL,000006.BKD...","885459.DDBK,885930.DDBK,885957.DDBK,885517.DDB...",980365.ZHBK,...,-68.31,-2.57,67,"{""9A0001"": {""realShortLineScore"": 40.07, ""real...","{""000012.BKDL"": {""categoryCode"": ""000012.BKDL""...",2025-02-28 17:22:47,0.008969,0.005969,0.066143,0.063143
7,2605,2025-02-14,低吸,中位低吸,002036.XSHE,联创电子,2,"000012.BKDL,000038.BKDL,000014.BKDL,000006.BKD...","886048.DDBK,885467.DDBK,885545.DDBK,885675.DDB...",980357.ZHBK,...,-74.52,-1.02,68,"{""9A0001"": {""realShortLineScore"": 27.53, ""real...","{""000003.BKDL"": {""categoryCode"": ""000003.BKDL""...",2025-02-28 17:23:40,0.091590,0.088590,0.105745,0.102745
8,2649,2025-02-17,低吸,中位低吸,002031.XSHE,巨轮智能,1,"000038.BKDL,000018.BKDL,000006.BKDL,000034.BKD...","885930.DDBK,885694.DDBK,885502.DDBK,885517.DDB...",980365.ZHBK,...,-54.96,12.25,67,"{""9A0001"": {""realShortLineScore"": 28.81, ""real...","{""000003.BKDL"": {""categoryCode"": ""000003.BKDL""...",2025-02-28 17:24:41,0.092105,0.089105,0.063158,0.060158
9,2714,2025-02-18,低吸,中位低吸,002908.XSHE,德生科技,1,"000012.BKDL,000003.BKDL,000038.BKDL,000010.BKD...","885975.DDBK,885757.DDBK,886041.DDBK,885629.DDB...","980753.ZHBK,980366.ZHBK,980376.ZHBK",...,-34.04,-2.67,57,"{""9A0001"": {""realShortLineScore"": 7.77, ""realT...","{""000004.BKDL"": {""categoryCode"": ""000004.BKDL""...",2025-02-28 17:25:40,-0.021761,-0.024761,0.046489,0.043489


In [4]:
combined_df

,id,date_key,strategy_name,sub_strategy_name,stock_code,stock_name,stock_rank,block_category,block_codes,industry_code,...,mod_name,mod_short_line_score,mod_short_line_score_change,mod_short_line_rank,mod_trend_score,mod_trend_score_change,mod_trend_rank,env_json_info,block_category_info,created_at
0,2233,2025-02-05,低吸,中位低吸,603211.XSHG,晋拓股份,1,"000027.BKDL,000007.BKDL,000004.BKDL,000033.BKDL","885467.DDBK,885517.DDBK,885929.DDBK,885832.DDB...",980372.ZHBK,...,中位低吸,-97.20,5.09,69,-83.43,-5.75,55,"{""9A0001"": {""realShortLineScore"": -2.23, ""real...","{""000034.BKDL"": {""categoryCode"": ""000034.BKDL""...",2025-02-28 17:17:10
1,2234,2025-02-05,低吸,中位低吸,002917.XSHE,金奥博,2,"000012.BKDL,000023.BKDL,000018.BKDL,000035.BKD...","885743.DDBK,885517.DDBK,885700.DDBK,885619.DDB...",980364.ZHBK,...,中位低吸,-97.20,5.09,69,-83.43,-5.75,55,"{""9A0001"": {""realShortLineScore"": -2.23, ""real...","{""000034.BKDL"": {""categoryCode"": ""000034.BKDL""...",2025-02-28 17:17:10
2,2235,2025-02-05,低吸,中位低吸,002265.XSHE,建设工业,3,"000038.BKDL,000027.BKDL,000013.BKDL,000017.BKD...","885743.DDBK,886021.DDBK,885694.DDBK,885595.DDB...",980372.ZHBK,...,中位低吸,-97.20,5.09,69,-83.43,-5.75,55,"{""9A0001"": {""realShortLineScore"": -2.23, ""real...","{""000034.BKDL"": {""categoryCode"": ""000034.BKDL""...",2025-02-28 17:17:10
3,2275,2025-02-06,低吸,中位低吸,002861.XSHE,瀛通通讯,1,"000012.BKDL,000014.BKDL,000006.BKDL,000027.BKD...","885785.DDBK,885774.DDBK,885800.DDBK,885934.DDB...",980357.ZHBK,...,中位低吸,-103.26,3.82,70,-96.17,-1.52,63,"{""9A0001"": {""realShortLineScore"": 35.17, ""real...","{""000031.BKDL"": {""categoryCode"": ""000031.BKDL""...",2025-02-28 17:18:08
4,2276,2025-02-06,低吸,中位低吸,002398.XSHE,垒知集团,2,"000012.BKDL,000003.BKDL,000008.BKDL,000036.BKD...","885692.DDBK,885617.DDBK,886019.DDBK,885942.DDB...","980343.ZHBK,980368.ZHBK",...,中位低吸,-103.26,3.82,70,-96.17,-1.52,63,"{""9A0001"": {""realShortLineScore"": 35.17, ""real...","{""000031.BKDL"": {""categoryCode"": ""000031.BKDL""...",2025-02-28 17:18:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,3135,2025-02-27,低吸,中位低吸,002164.XSHE,宁波东力,4,"000035.BKDL,000004.BKDL,000009.BKDL,000033.BKDL","885924.DDBK,885993.DDBK,885641.DDBK,886008.DDB...",980365.ZHBK,...,中位低吸,126.09,38.14,11,42.13,0.33,16,"{""9A0001"": {""realShortLineScore"": 43.12, ""real...","{""000027.BKDL"": {""categoryCode"": ""000027.BKDL""...",2025-02-28 17:32:56
80,3136,2025-02-27,低吸,中位低吸,000678.XSHE,襄阳轴承,5,"000015.BKDL,000027.BKDL,000013.BKDL,000033.BKDL","885743.DDBK,886008.DDBK,885835.DDBK,885431.DDB...",980372.ZHBK,...,中位低吸,126.09,38.14,11,42.13,0.33,16,"{""9A0001"": {""realShortLineScore"": 43.12, ""real...","{""000027.BKDL"": {""categoryCode"": ""000027.BKDL""...",2025-02-28 17:32:56
81,3182,2025-02-28,低吸,中位低吸,000892.XSHE,欢瑞世纪,1,"000012.BKDL,000010.BKDL","885420.DDBK,885788.DDBK,886060.DDBK,885890.DDB...","980355.ZHBK,980348.ZHBK",...,中位低吸,117.85,-17.19,12,40.81,-6.17,16,"{""9A0001"": {""realShortLineScore"": -20.48, ""rea...","{""000033.BKDL"": {""categoryCode"": ""000033.BKDL""...",2025-02-28 17:33:57
82,3183,2025-02-28,低吸,中位低吸,002522.XSHE,浙江众成,2,"000036.BKDL,000006.BKDL,000004.BKDL,000022.BKD...","885924.DDBK,886021.DDBK,885806.DDBK,885929.DDB...","980364.ZHBK,980373.ZHBK",...,中位低吸,117.85,-17.19,12,40.81,-6.17,16,"{""9A0001"": {""realShortLineScore"": -20.48, ""rea...","{""000033.BKDL"": {""categoryCode"": ""000033.BKDL""...",2025-02-28 17:33:57


In [5]:
test_df = combined_df[combined_df['date_key'] == '2025-02-11']

In [6]:
test_df

,id,date_key,strategy_name,sub_strategy_name,stock_code,stock_name,stock_rank,block_category,block_codes,industry_code,...,mod_name,mod_short_line_score,mod_short_line_score_change,mod_short_line_rank,mod_trend_score,mod_trend_score_change,mod_trend_rank,env_json_info,block_category_info,created_at
18,2452,2025-02-11,低吸,中位低吸,603110.XSHG,东方材料,1,"000003.BKDL,000023.BKDL,000031.BKDL,000037.BKD...","885869.DDBK,885520.DDBK,885887.DDBK,885739.DDB...",980364.ZHBK,...,中位低吸,-12.19,50.1,68,-73.76,16.84,68,"{""9A0001"": {""realShortLineScore"": 54.81, ""real...","{""000003.BKDL"": {""categoryCode"": ""000003.BKDL""...",2025-02-28 17:20:53
19,2453,2025-02-11,低吸,中位低吸,002213.XSHE,大为股份,2,"000003.BKDL,000026.BKDL,000035.BKDL,000015.BKD...","885840.DDBK,886042.DDBK,885517.DDBK,885675.DDB...","980372.ZHBK,980357.ZHBK,980352.ZHBK",...,中位低吸,-12.19,50.1,68,-73.76,16.84,68,"{""9A0001"": {""realShortLineScore"": 54.81, ""real...","{""000003.BKDL"": {""categoryCode"": ""000003.BKDL""...",2025-02-28 17:20:53
20,2454,2025-02-11,低吸,中位低吸,603150.XSHG,万朗磁塑,3,"000036.BKDL,000026.BKDL,000018.BKDL,000015.BKD...","885840.DDBK,885901.DDBK,885946.DDBK,885343.DDB...",980364.ZHBK,...,中位低吸,-12.19,50.1,68,-73.76,16.84,68,"{""9A0001"": {""realShortLineScore"": 54.81, ""real...","{""000003.BKDL"": {""categoryCode"": ""000003.BKDL""...",2025-02-28 17:20:53
21,2455,2025-02-11,低吸,中位低吸,001368.XSHE,通达创智,4,"000021.BKDL,000012.BKDL,000026.BKDL,000018.BKD...","885899.DDBK,885840.DDBK,885598.DDBK,885617.DDB...",980373.ZHBK,...,中位低吸,-12.19,50.1,68,-73.76,16.84,68,"{""9A0001"": {""realShortLineScore"": 54.81, ""real...","{""000003.BKDL"": {""categoryCode"": ""000003.BKDL""...",2025-02-28 17:20:53
22,2456,2025-02-11,低吸,中位低吸,002861.XSHE,瀛通通讯,5,"000012.BKDL,000014.BKDL,000006.BKDL,000027.BKD...","885785.DDBK,885774.DDBK,885800.DDBK,885934.DDB...",980357.ZHBK,...,中位低吸,-12.19,50.1,68,-73.76,16.84,68,"{""9A0001"": {""realShortLineScore"": 54.81, ""real...","{""000003.BKDL"": {""categoryCode"": ""000003.BKDL""...",2025-02-28 17:20:53
